In [1]:
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import find_peaks
from scipy.fft import fft, ifft, fftfreq
from scipy import signal
import statistics as st
import warnings
import sys
import importlib.util
import os
# Suppress all warnings
warnings.filterwarnings('ignore')

C:\Users\User\Anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
current_dir = os.path.dirname(os.path.abspath('__file__'))

# Define the path to the functions directory
functions_dir = os.path.join(current_dir, 'functions')

# Add the functions directory to sys.path
sys.path.append(functions_dir)
from PreProcessing  import PreProcessing 
# import features
from gaitEvents import gaitEvent
# import frequency
from frequency import frequencyDomain
from features import gaitFeatures

In [3]:
freq=frequencyDomain()
pre_processor = PreProcessing()


## Reading all csv to one dictionary

In [4]:

# Function to read all patient data
def read_all_patient_data(base_path):
    patient_data = {}
    for patient_dir in os.listdir(base_dir):
        patient_path = os.path.join(base_dir, patient_dir)
        print(patient_dir)
        if os.path.isdir(patient_path):
            patient_data[patient_dir] = []
           
            for trial_dir in os.listdir(patient_path):
                trial_path = os.path.join(patient_path, trial_dir)
                df =pre_processor.read_csv_file(trial_path) 
                if len(df.columns) > 14 :
                    df = df.iloc[:, :14]
                    df.columns = ['time', 'knee_angle', 'accX_T', 'accY_T', 'accZ_T', 'gyrX_T', 'gyrY_T', 'gyrZ_T', 'accX_C', 'accY_C', 'accZ_C', 'gyrX_C', 'gyrY_C', 'gyrZ_C']
                else:
                    df.columns = ['time', 'knee_angle', 'accX_T', 'accY_T', 'accZ_T', 'gyrX_T', 'gyrY_T', 'gyrZ_T', 'accX_C', 'accY_C', 'accZ_C', 'gyrX_C', 'gyrY_C', 'gyrZ_C']
                patient_data[patient_dir].append(df)
    return patient_data

def handleNan(patient_data):
    filtered_data = {}
    for patient_id in patients_data:
        filtered_data[patient_id] = []
        for trail_df in patients_data[patient_id]:
            trail_df = trail_df.dropna()
            trail_df = trail_df.reset_index(drop=True)
            filtered_data[patient_id].append(trail_df)
    return filtered_data

In [5]:
# Define the path to the directory containing patient folders
base_dir = 'csv/data/'
# Read all patient data
patients_data = read_all_patient_data(base_dir)
patients_data["1"][0]

1
10
11
12
13
14
15
16
17
18
19
2
20
21
22
23
3
4
5
6
7
8
9


,time,knee_angle,accX_T,accY_T,accZ_T,gyrX_T,gyrY_T,gyrZ_T,accX_C,accY_C,accZ_C,gyrX_C,gyrY_C,gyrZ_C
0,1101.22,71.21,-9.7,-1.05,2.52,-0.04,-0.02,0.00,-0.75,0.15,10.03,0.01,0.0,0.01
1,1101.23,71.21,-9.7,-1.04,2.52,-0.04,-0.02,0.00,-0.75,0.16,10.03,0.01,0.0,0.00
2,1101.24,71.21,-9.7,-1.04,2.52,-0.04,-0.02,0.00,-0.75,0.16,10.04,0.01,0.0,0.00
3,1101.26,71.21,-9.7,-1.04,2.51,-0.04,-0.02,0.00,-0.76,0.17,10.03,0.01,0.0,0.00
4,1101.27,71.21,-9.71,-1.04,2.51,-0.04,-0.02,0.00,-0.76,0.17,10.04,0.01,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1413,1120.92,62.78,-9.27,0.15,4.14,-0.05,-0.02,-0.01,-0.52,0.26,10.04,0.02,0.0,0.00
1414,1120.93,62.79,-9.22,0.17,4.14,-0.05,-0.03,-0.01,-0.49,0.28,10.02,0.02,0.0,0.00
1415,1120.94,62.81,-9.17,0.18,4.12,-0.05,-0.03,-0.01,-0.48,0.28,10.00,0.03,0.0,0.00
1416,1120.96,62.81,-9.12,0.18,4.14,-0.06,-0.03,-0.01,-0.47,0.28,9.97,0.03,0.0,0.00


In [6]:
for i in patients_data:
    print(i,"contain", len(patients_data[i]), "files")  

df_f = handleNan(patients_data)
# filtered_data["9"][2]

1 contain 3 files
10 contain 3 files
11 contain 5 files
12 contain 4 files
13 contain 4 files
14 contain 4 files
15 contain 4 files
16 contain 4 files
17 contain 4 files
18 contain 4 files
19 contain 3 files
2 contain 3 files
20 contain 4 files
21 contain 4 files
22 contain 4 files
23 contain 4 files
3 contain 3 files
4 contain 2 files
5 contain 3 files
6 contain 4 files
7 contain 4 files
8 contain 4 files
9 contain 3 files


# Main "Time & frequency domain features dataframe"

In [14]:

patient_trail_features = []

for patient_id in df_f:
    print("P",patient_id)
    for trial_df in df_f[patient_id]:
        negative_count = (trial_df['knee_angle'] < 0).sum()
        if(negative_count> 200):
            trial_df.iloc[:, 1:] = trial_df.iloc[:, 1:] * -1
        first_peak_min_index, last_peak_min_index =pre_processor.FirstLastIndxs(trial_df['knee_angle'])
        ################################################### Select moving part ############################################# 
        # select the interval starting from first min peak to the last min peak 
        df_moving = trial_df.iloc[first_peak_min_index:last_peak_min_index + 1]
        df_moving = df_moving.reset_index(drop=True)
        df_moving["time"] = pd.to_numeric(df_moving["time"], errors = 'coerce')
        ##################################################  Filter ########################################################
        idxHS = np.asarray([], dtype=int)
        idxTO = np.asarray([], dtype=int)
        fcgyr = 12  # Cut-off frequency of the filter for gyroscope data
        fcacc = 17 # Cut-off frequency of the filter for accelerometer data
        sample_frequency=100
        N = 2 # Order of the butterworth filter
        hs_angles=[]
        to_angles=[]

        try:
            fGYRyR =pre_processor.Filter(fcgyr, sample_frequency, df_moving["gyrY_C"], 'lowpass', N)
            fcaccR = pre_processor.Filter(fcacc, sample_frequency, df_moving["accZ_C"], 'lowpass', N)
            faccYC =pre_processor.Filter(fcacc,sample_frequency,df_moving["accY_C"],'lowpass',N)
            faccYT = pre_processor.Filter(fcacc,sample_frequency,df_moving["accY_T"],'lowpass',N)
        except Exception as e:
            print("Filtering Error:", e)
        ####################### Mid stance / Heal strike / Toe off points ##########################################
        idxOnsMSw =gaitEvent.MidStance(fGYRyR,0.2,0.2,100)
        idxHS =gaitEvent.heelStrike(idxOnsMSw ,fGYRyR)
        idxTO =gaitEvent.toeOff(idxOnsMSw,fcaccR,fGYRyR,100)
        swing_time_diff = gaitFeatures.functionswingtime(idxTO, idxHS, 100, np.array(df_moving["time"]))
        stance_time_diff = gaitFeatures.functionstancetime(idxTO, idxHS, 100,  np.array(df_moving["time"]))
        stride_time_diff = gaitFeatures.functionstridetime(idxHS, 100, np.array(df_moving["time"]))
        gait_kinematics = pre_processor.SwingStanceStrideTimeDiffMaxMin(swing_time_diff, stance_time_diff, stride_time_diff,idxHS)
        ################################################# Angles Calculation ##################################################
        angle_features = []
        df2 = trial_df.iloc[:first_peak_min_index]
        setting_angle = df2["knee_angle"].max()
        angle_features.append(setting_angle)
        for hs_idx, to_idx in zip(idxHS, idxTO):   
            hs_angles.append(df_moving["knee_angle"][hs_idx])
            to_angles.append(df_moving["knee_angle"][to_idx])
        mean_hs_angle = np.mean(hs_angles)
        angle_features.append(mean_hs_angle)
        mean_to_angle = np.mean(to_angles)
        angle_features.append(mean_to_angle)
        max_stance_angles, max_swing_angles = gaitFeatures.MaxAngle(df_moving["knee_angle"],idxTO,idxHS)
        angle_features.append( np.mean(max_stance_angles))
        angle_features.append( np.mean(max_swing_angles))
        ################################################ peak to peak acc Calculation ##################################################
        lowerPeaksClaf,  higherPeaksClaf = gaitFeatures.accPeaks(faccYC, df_moving['time'],idxHS)
        lowerPeaksThigh, higherPeaksThigh = gaitFeatures.accPeaks(faccYT, df_moving['time'],idxHS)
        thrustACCel_C = gaitFeatures.ThrustAcc(higherPeaksClaf, lowerPeaksClaf)
        angle_features.append( np.mean(thrustACCel_C))
        thrustACCel_T = gaitFeatures.ThrustAcc(higherPeaksThigh, lowerPeaksThigh)
        angle_features.append( np.mean(thrustACCel_T))
         ################################################### Frequency features ############################################# 
        acc_axis_name_T = ['accX_T', 'accY_T', 'accZ_T']
        acc_axis_name_C = ['accX_C', 'accY_C', 'accZ_C']
        Fs = freq.Sampling_freq(df_moving["time"])
        num_cycles = freq.calculate_number_of_cycles(df_moving["time"], 6)
        omega_features_T, power_features_T =freq. FrequencyFeaturesArr(df_moving, acc_axis_name_T,100,num_cycles)
        omega_features_C, power_features_C = freq.FrequencyFeaturesArr(df_moving, acc_axis_name_C,100,num_cycles)
        # append data
        patient_trail_features.append([
            patient_id,
            *gait_kinematics ,
            *angle_features,
            *omega_features_T,
            *(power_features_T[axis][i] for axis in range(3) for i in range(6)),
            *omega_features_C,
            *(power_features_C[axis][i] for axis in range(3) for i in range(6))
        ])
    

column_names = ['patient_id', 'swing_time_max', 'swing_time_min', 'stance_time_max', 'stance_time_min',
                'stride_time_max', 'stride_time_min',"setting_angle" , 'mean_hs_angle', 'mean_to_angle',
                'max_stance_angles', 'max_swing_angles','thrustACCel_C', 'thrustACCel_T', 
                "Omega_X_T", "Omega_Y_T", "Omega_Z_T", 
                "powerX_ 1_T", "powerX_ 2_T", "powerX_ 3_T", "powerX_ 4_T", "powerX_ 5_T", "powerX_ 6_T", 
                "powerY_ 1_T", "powerY_ 2_T","powerY_ 3_T","powerY_ 4_T", "powerY_ 5_T","powerY_ 6_T",
                "powerZ_ 1_T", "powerZ_ 2_T", "powerZ_ 3_T", "powerZ_ 4_T", "powerZ_ 5_T", "powerZ_ 6_T",
                "Omega_X_C", "Omega_Y_C", "Omega_Z_C", 
                "powerX_ 1_C", "powerX_ 2_C", "powerX_ 3_C", "powerX_ 4_C", "powerX_ 5_C", "powerX_ 6_C", 
                "powerY_ 1_C", "powerY_ 2_T","powerY_ 3_C","powerY_ 4_C", "powerY_ 5_C","powerY_ 6_C",
                "powerZ_ 1_C", "powerZ_ 2_C", "powerZ_ 3_C", "powerZ_ 4_C", "powerZ_ 5_C", "powerZ_ 6_C"
               ]
df_patient_trail_features = pd.DataFrame(patient_trail_features,columns=column_names)
df_patient_trail_features.head()

P 1
P 10
P 11
P 12
P 13
P 14
P 15
P 16
P 17
P 18
P 19
P 2
P 20
P 21
P 22
P 23
P 3
P 4
P 5
P 6
P 7
P 8
P 9


,patient_id,swing_time_max,swing_time_min,stance_time_max,stance_time_min,stride_time_max,stride_time_min,setting_angle,mean_hs_angle,mean_to_angle,...,powerY_ 3_C,powerY_ 4_C,powerY_ 5_C,powerY_ 6_C,powerZ_ 1_C,powerZ_ 2_C,powerZ_ 3_C,powerZ_ 4_C,powerZ_ 5_C,powerZ_ 6_C
0,1,1.04,0.46,1.18,0.81,1.99,1.31,71.29,24.000000,38.840000,...,1040.211971,73.481629,141.905164,10.216969,1.571939e+06,607.747259,533.161447,2633.837158,3055.918834,173.842186
1,1,1.22,0.42,1.39,0.08,2.24,0.53,64.05,21.227143,34.982857,...,1964.301772,816.673624,404.261715,228.916362,2.954191e+06,109.416956,2606.851615,10.729508,5185.243035,25.271313
2,1,1.01,0.42,1.01,0.23,2.02,0.65,67.16,17.451667,32.843333,...,4199.071301,3488.940407,113.009892,760.522046,1.902485e+06,860.193745,304.164403,12119.034787,1532.932283,537.452090
3,10,0.90,0.43,0.85,0.52,1.61,1.00,71.26,11.700000,27.365000,...,753.474303,535.838457,105.252777,182.907451,9.738762e+05,2350.306948,2954.338341,2614.545432,1978.834077,105.690189
4,10,0.61,0.36,0.86,0.34,1.43,0.95,69.07,17.008571,28.935714,...,442.169719,431.027909,176.619568,75.768927,1.519685e+06,72.975955,462.167270,1289.025156,711.321877,26.048736


In [15]:
df_patient_trail_features

,patient_id,swing_time_max,swing_time_min,stance_time_max,stance_time_min,stride_time_max,stride_time_min,setting_angle,mean_hs_angle,mean_to_angle,...,powerY_ 3_C,powerY_ 4_C,powerY_ 5_C,powerY_ 6_C,powerZ_ 1_C,powerZ_ 2_C,powerZ_ 3_C,powerZ_ 4_C,powerZ_ 5_C,powerZ_ 6_C
0,1,1.04,0.46,1.18,0.81,1.99,1.31,71.29,24.000000,38.840000,...,1040.211971,73.481629,141.905164,10.216969,1.571939e+06,607.747259,533.161447,2633.837158,3055.918834,173.842186
1,1,1.22,0.42,1.39,0.08,2.24,0.53,64.05,21.227143,34.982857,...,1964.301772,816.673624,404.261715,228.916362,2.954191e+06,109.416956,2606.851615,10.729508,5185.243035,25.271313
2,1,1.01,0.42,1.01,0.23,2.02,0.65,67.16,17.451667,32.843333,...,4199.071301,3488.940407,113.009892,760.522046,1.902485e+06,860.193745,304.164403,12119.034787,1532.932283,537.452090
3,10,0.90,0.43,0.85,0.52,1.61,1.00,71.26,11.700000,27.365000,...,753.474303,535.838457,105.252777,182.907451,9.738762e+05,2350.306948,2954.338341,2614.545432,1978.834077,105.690189
4,10,0.61,0.36,0.86,0.34,1.43,0.95,69.07,17.008571,28.935714,...,442.169719,431.027909,176.619568,75.768927,1.519685e+06,72.975955,462.167270,1289.025156,711.321877,26.048736
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,8,0.70,0.47,1.15,0.82,1.85,1.31,68.90,16.812000,25.746000,...,789.310679,1729.381985,374.661370,99.057454,1.052067e+06,664.417656,1520.306683,1620.084514,810.152477,98.917555
80,8,2.39,0.52,1.38,0.59,2.98,1.36,69.19,18.786667,22.300000,...,277.287383,481.054252,1206.120977,173.529970,1.912495e+06,164.905455,45.824601,1505.276923,991.774908,18.398063
81,9,1.35,0.49,1.23,0.57,2.19,1.27,68.09,24.932000,20.004000,...,3719.174305,260.874791,3482.132657,101.524289,3.668234e+06,90.113790,2444.744831,38.121232,1453.824808,100.249514
82,9,1.19,0.38,1.25,0.59,1.78,1.09,62.75,22.143333,18.550000,...,360.914764,165.198159,777.788261,465.647355,1.355754e+06,599.345429,681.342965,385.500887,252.893451,112.816790


In [9]:
# df_patient_trail_features.to_csv("csv/data_training/AllFeatures_true.csv", index=False)

# frequency domain features dataframe only

In [10]:

patient_trail_features = []

for patient_id in df_f: 
    for trail_df in df_f[patient_id]:
        negative_count = (trial_df['knee_angle'] < 0).sum()
        if(negative_count> 200):
            trial_df.iloc[:, 1:] = trial_df.iloc[:, 1:] * -1

        first_peak_min_index, last_peak_min_index =pre_processor.FirstLastIndxs(trial_df['knee_angle'])
        ################################################### Select moving part ############################################# 
        # select the interval starting from first min peak to the last min peak 
        df_moving = trial_df.iloc[first_peak_min_index:last_peak_min_index + 1]
        df_moving = df_moving.reset_index(drop=True)
        df_moving["time"] = pd.to_numeric(df_moving["time"], errors = 'coerce')
         ################################################### Frequency features ############################################# 
        acc_axis_name_T = ['accX_T', 'accY_T', 'accZ_T']
        Fs = freq.Sampling_freq(df_moving["time"])
        num_cycles = freq.calculate_number_of_cycles(df_moving["time"], 6)
        omega_features_T, power_features_T =freq. FrequencyFeaturesArr(df_moving, acc_axis_name_T,100,num_cycles)

        # append data
        patient_trail_features.append([
            patient_id,
            *omega_features_T,
            *(power_features_T[axis][i] for axis in range(3) for i in range(6)),
        ])
    
column_names = ['patient_id',
                "Omega_X", "Omega_Y", "Omega_Z", 
                "powerX_ 1", "powerX_ 2", "powerX_ 3", "powerX_ 4", "powerX_ 5", "powerX_ 6", 
                "powerY_ 1", "powerY_ 2","powerY_ 3","powerY_ 4", "powerY_ 5","powerY_ 6",
                "powerZ_ 1", "powerZ_ 2", "powerZ_ 3", "powerZ_ 4", "powerZ_ 5", "powerZ_ 6"]

df_patient_trail_features = pd.DataFrame(patient_trail_features, columns=column_names)
df_patient_trail_features.head()

,patient_id,Omega_X,Omega_Y,Omega_Z,powerX_ 1,powerX_ 2,powerX_ 3,powerX_ 4,powerX_ 5,powerX_ 6,...,powerY_ 3,powerY_ 4,powerY_ 5,powerY_ 6,powerZ_ 1,powerZ_ 2,powerZ_ 3,powerZ_ 4,powerZ_ 5,powerZ_ 6
0,1,0.960219,0.548697,0.274348,193802.4529,11316.198287,536.806984,8410.153704,5280.725645,1736.445254,...,230.941386,659.303239,206.164771,72.959118,1.035313e+06,11189.318896,2319.879416,1508.356505,288.946573,237.09342
1,1,0.960219,0.548697,0.274348,193802.4529,11316.198287,536.806984,8410.153704,5280.725645,1736.445254,...,230.941386,659.303239,206.164771,72.959118,1.035313e+06,11189.318896,2319.879416,1508.356505,288.946573,237.09342
2,1,0.960219,0.548697,0.274348,193802.4529,11316.198287,536.806984,8410.153704,5280.725645,1736.445254,...,230.941386,659.303239,206.164771,72.959118,1.035313e+06,11189.318896,2319.879416,1508.356505,288.946573,237.09342
3,10,0.960219,0.548697,0.274348,193802.4529,11316.198287,536.806984,8410.153704,5280.725645,1736.445254,...,230.941386,659.303239,206.164771,72.959118,1.035313e+06,11189.318896,2319.879416,1508.356505,288.946573,237.09342
4,10,0.960219,0.548697,0.274348,193802.4529,11316.198287,536.806984,8410.153704,5280.725645,1736.445254,...,230.941386,659.303239,206.164771,72.959118,1.035313e+06,11189.318896,2319.879416,1508.356505,288.946573,237.09342


In [11]:
# df_patient_trail_features.to_csv("csv/data_training/one-domain-only/freq_FeaturesT.csv", index=False)

# time alone or freq alone

In [17]:
feature = "t"
max_value_features = []
min_value_features = []
patient_trail_features = []
    


if feature == "f":
    for patient_id in df_f: 
        for trial_df in df_f[patient_id]:
            negative_count = (trial_df['knee_angle'] < 0).sum()
            if(negative_count> 200):
                print("y",patient_id)
                trial_df.iloc[:, 1:] = trial_df.iloc[:, 1:] * -1
            first_peak_min_index, last_peak_min_index =pre_processor.FirstLastIndxs(trial_df['knee_angle'])
            ################################################### Select moving part ############################################# 
            # select the interval starting from first min peak to the last min peak 
            df_moving = trial_df.iloc[first_peak_min_index:last_peak_min_index + 1]
            df_moving = df_moving.reset_index(drop=True)
            df_moving["time"] = pd.to_numeric(df_moving["time"], errors = 'coerce')
            ################################################### Frequency features ############################################# 
            acc_axis_name_T = ['accX_T', 'accY_T', 'accZ_T']
            acc_axis_name_C = ['accX_C', 'accY_C', 'accZ_C']
            Fs = freq.Sampling_freq(df_moving["time"])
            num_cycles = freq.calculate_number_of_cycles(df_moving["time"], 6)
            omega_features_T, power_features_T =freq. FrequencyFeaturesArr(df_moving, acc_axis_name_T,100,num_cycles)
            omega_features_C, power_features_C = freq.FrequencyFeaturesArr(df_moving, acc_axis_name_C,100,num_cycles)
            # append data
            patient_trail_features.append([
                patient_id,
                *omega_features_T,
                *(power_features_T[axis][i] for axis in range(3) for i in range(6)),
                *omega_features_C,
                *(power_features_C[axis][i] for axis in range(3) for i in range(6))
            ])

    column_names = ['patient_id', 
                "Omega_X_T", "Omega_Y_T", "Omega_Z_T", 
                "powerX_ 1_T", "powerX_ 2_T", "powerX_ 3_T", "powerX_ 4_T", "powerX_ 5_T", "powerX_ 6_T", 
                "powerY_ 1_T", "powerY_ 2_T","powerY_ 3_T","powerY_ 4_T", "powerY_ 5_T","powerY_ 6_T",
                "powerZ_ 1_T", "powerZ_ 2_T", "powerZ_ 3_T", "powerZ_ 4_T", "powerZ_ 5_T", "powerZ_ 6_T",
                "Omega_X_C", "Omega_Y_C", "Omega_Z_C", 
                "powerX_ 1_C", "powerX_ 2_C", "powerX_ 3_C", "powerX_ 4_C", "powerX_ 5_C", "powerX_ 6_C", 
                "powerY_ 1_C", "powerY_ 2_T","powerY_ 3_C","powerY_ 4_C", "powerY_ 5_C","powerY_ 6_C",
                "powerZ_ 1_C", "powerZ_ 2_C", "powerZ_ 3_C", "powerZ_ 4_C", "powerZ_ 5_C", "powerZ_ 6_C"
               ]
    
    df_freq_features = pd.DataFrame(patient_trail_features, columns=column_names)
    
    df_freq_features.to_csv("csv/data_training/one-domain-only/freq.csv", index=False)
else:
    for patient_id in df_f: 
        for trial_df in df_f[patient_id]:
            negative_count = (trial_df['knee_angle'] < 0).sum()
            if(negative_count> 200):
                trial_df.iloc[:, 1:] = trial_df.iloc[:, 1:] * -1
            first_peak_min_index, last_peak_min_index =pre_processor.FirstLastIndxs(trial_df['knee_angle'])
            ################################################### Select moving part ############################################# 
            # select the interval starting from first min peak to the last min peak 
            df_moving = trial_df.iloc[first_peak_min_index:last_peak_min_index + 1]
            df_moving = df_moving.reset_index(drop=True)
            df_moving["time"] = pd.to_numeric(df_moving["time"], errors = 'coerce')
            ##################################################  Filter ########################################################
            idxHS = np.asarray([], dtype=int)
            idxTO = np.asarray([], dtype=int)
            fcgyr = 12  # Cut-off frequency of the filter for gyroscope data
            fcacc = 17 # Cut-off frequency of the filter for accelerometer data
            sample_frequency=100
            N = 2 # Order of the butterworth filter
            hs_angles=[]
            to_angles=[]

            try:
                fGYRyR =pre_processor.Filter(fcgyr, sample_frequency, df_moving["gyrY_C"], 'lowpass', N)
                fcaccR = pre_processor.Filter(fcacc, sample_frequency, df_moving["accZ_C"], 'lowpass', N)
                faccYC =pre_processor.Filter(fcacc,sample_frequency,df_moving["accY_C"],'lowpass',N)
                faccYT = pre_processor.Filter(fcacc,sample_frequency,df_moving["accY_T"],'lowpass',N)
            except Exception as e:
                print("Filtering Error:", e)
            ####################### Mid stance / Heal strike / Toe off points ##########################################
            idxOnsMSw =gaitEvent.MidStance(fGYRyR,0.2,0.2,100)
            idxHS =gaitEvent.heelStrike(idxOnsMSw ,fGYRyR)
            idxTO =gaitEvent.toeOff(idxOnsMSw,fcaccR,fGYRyR,100)
            swing_time_diff = gaitFeatures.functionswingtime(idxTO, idxHS, 100, np.array(df_moving["time"]))
            stance_time_diff = gaitFeatures.functionstancetime(idxTO, idxHS, 100,  np.array(df_moving["time"]))
            stride_time_diff = gaitFeatures.functionstridetime(idxHS, 100, np.array(df_moving["time"]))
            gait_kinematics = pre_processor.SwingStanceStrideTimeDiffMaxMin(swing_time_diff, stance_time_diff, stride_time_diff,idxHS)
            ################################################# Angles Calculation ##################################################
            angle_features = []
            df2 = trial_df.iloc[:first_peak_min_index]
            setting_angle = df2["knee_angle"].max()
            angle_features.append(setting_angle)
            for hs_idx, to_idx in zip(idxHS, idxTO):   
                hs_angles.append(df_moving["knee_angle"][hs_idx])
                to_angles.append(df_moving["knee_angle"][to_idx])
            mean_hs_angle = np.mean(hs_angles)
            angle_features.append(mean_hs_angle)
            mean_to_angle = np.mean(to_angles)
            angle_features.append(mean_to_angle)
            max_stance_angles, max_swing_angles = gaitFeatures.MaxAngle(df_moving["knee_angle"],idxTO,idxHS)
            angle_features.append( np.mean(max_stance_angles))
            angle_features.append( np.mean(max_swing_angles))
            ################################################ peak to peak acc Calculation ##################################################
            lowerPeaksClaf,  higherPeaksClaf = gaitFeatures.accPeaks(faccYC, df_moving['time'],idxHS)
            lowerPeaksThigh, higherPeaksThigh = gaitFeatures.accPeaks(faccYT, df_moving['time'],idxHS)
            thrustACCel_C = gaitFeatures.ThrustAcc(higherPeaksClaf, lowerPeaksClaf)
            angle_features.append( np.mean(thrustACCel_C))
            thrustACCel_T = gaitFeatures.ThrustAcc(higherPeaksThigh, lowerPeaksThigh)
            angle_features.append( np.mean(thrustACCel_T))

            patient_trail_features.append([
                patient_id,
                *gait_kinematics ,
                *angle_features
            ])
    column_names = ['patient_id', 'swing_time_max', 'swing_time_min', 'stance_time_max', 'stance_time_min',
                    'stride_time_max', 'stride_time_min', 'setting_angle', 'mean_hs_angle', 'mean_to_angle',
                    'thrustACCel_C', 'thrustACCel_T', 'max_stance_angles', 'max_swing_angles']


    df_time_features = pd.DataFrame(patient_trail_features, columns=column_names)
    df_time_features.to_csv("csv/data_training/one-domain-only/time_FeaturesT.csv", index=False)
